## KDJ 当日金叉 死叉

In [1]:
import akshare as ak
from datetime import datetime, timedelta
from pathlib import Path

import common
import calculate

In [2]:
file_name = "./txt_lib/stock_code.txt"
with open(file_name, "r") as file:
    stock_list = [line.strip() for line in file if line.strip()]

# # 限制股票数量
# stock_list = stock_list[:150]

# all_cross_dates = process_stocks(stock_list)
all_cross_dates = common.Final_process_KDJ.process_stocks(stock_list)

In [3]:
try:
    folder = Path("./output/golden_output")
    folder.mkdir()
    folder = Path("./output/death_output")
    folder.mkdir()
    folder = Path("./output/操作1.1")
    folder.mkdir()
except Exception as e:
    pass

# 清理两个文件夹
common.Final_file.clean_folder("./output/death_output")
common.Final_file.clean_folder("./output/golden_output")
common.Final_file.clean_folder("./output/操作1.1")

In [4]:
# 选出今日金叉list
today_gold_cross_list = []
for stock_code in all_cross_dates.keys():
    if all_cross_dates[stock_code]["Golden_Cross"]:
        today_gold_cross_list.append(stock_code)

# 选出今日死叉list
today_death_cross_list = []
for stock_code in all_cross_dates.keys():
    if all_cross_dates[stock_code]["Death_Cross"]:
        today_death_cross_list.append(stock_code)

# KDJ日金 + MACD 2 条件 → 操作1.1
to_perfect_stock_list = common.Final_process.process_stocks(
    today_gold_cross_list, mode="MACD_2_condition"
)
# output_file = "./output/操作1.1/操作1.1.xlsx"
# common.Final_file.output_excel(to_perfect_stock_list, output_file, condition="完美")
# common.Final_file.mk_pic(output_file)  # 生成所有图片

In [5]:
# 生成今日金叉 死叉Excel
common.Final_file_KDJ.output_excel(all_cross_dates)

# 生成今日金叉 死叉TXT
print("已生成今日金叉 死叉TXT")
today_death_txt_path = "./output/death_output/today_death_cross.txt"
with open(today_death_txt_path, "w") as file:
    for item in today_death_cross_list:
        file.write(item + "\n")  # 每个元素后加换行符

today_gold_txt_path = "./output/golden_output/today_gold_cross.txt"
with open(today_gold_txt_path, "w") as file:
    for item in today_gold_cross_list:
        file.write(item + "\n")  # 每个元素后加换行符

Excel 文件已生成
已生成今日金叉 死叉TXT


# 按需分流 KDJ日金叉 & KDJ日死叉 代码仓库

In [6]:
# 给原始股票代码加上抬头；'600001' → 'sh600001'
def add_capital(input_file: str, output_file: str):
    with open(input_file, "r") as infile, open(output_file, "w") as outfile:
        for line in infile:
            stock_code = line.strip()
            if stock_code.startswith("6"):
                processed_code = f"sh{stock_code}"
            else:
                processed_code = f"sz{stock_code}"
            outfile.write(processed_code + "\n")

    print(f"Processed stock codes saved to {output_file}.")

# 复制一份昨日txt 留档
def copy_yest_txt(input_file: str, output_file: str):
    with open(input_file, "r") as infile, open(output_file, "w") as outfile:
        for line in infile:
            stock_code = line.strip()
            outfile.write(stock_code + "\n")
    print(f"Processed stock codes saved to {output_file}.")


dict_filename = "./txt_lib/stock_name.txt"
stock_dict = common.Initialization.generate_stock_dict(dict_filename)

input_file = "./txt_lib/KDJ日金.txt"
output_file = "./txt_lib/yest_KDJ日金.txt"
copy_yest_txt(input_file,output_file)
input_file = "./txt_lib/KDJ日死.txt"
output_file = "./txt_lib/yest_KDJ日死.txt"
copy_yest_txt(input_file, output_file)

# 清理两个文件夹
common.Final_file.clean_folder("./output/KDJ日金")
common.Final_file.clean_folder("./output/KDJ日死")
common.Final_file.clean_folder("./output/KDJ8")

Processed stock codes saved to ./txt_lib/yest_KDJ日金.txt.
Processed stock codes saved to ./txt_lib/yest_KDJ日死.txt.


In [29]:
## 处理 KDJ日金 + 周死叉 → 弃
# 读取原始文件并处理
input_file = "./txt_lib/KDJ日金.txt"
output_file = "./txt_lib/processed_KDJ日金.txt"
add_capital(input_file, output_file)

file_name = output_file
with open(file_name, "r") as file:
    gold_stock_list = [line.strip() for line in file if line.strip()]

gold_to_del_stock_list = []
gold_to_del_week_death = common.Final_process.process_stocks(gold_stock_list, mode="week_death")

## 处理 KDJ日金 + macd < 0 → 弃

gold_macd_below0 = []
gold_macd_below0 = common.Final_process.process_stocks(gold_stock_list, mode="macd < 0")

## 处理 KDJ日金 + J < K → 弃
gold_J_K = []
gold_J_K = common.Final_process.process_stocks(gold_stock_list, mode="J_K")

# 把选出股票加入 to_del 数组中
gold_to_del_stock_list = gold_to_del_week_death + gold_macd_below0 + gold_J_K

Processed stock codes saved to ./txt_lib/processed_KDJ日金.txt.


In [8]:
## 处理 KDJ日死 + 周死叉 → 弃
# 读取原始文件并处理
input_file = "./txt_lib/KDJ日死.txt"
output_file = "./txt_lib/processed_KDJ日死.txt"
add_capital(input_file, output_file)

file_name = output_file
with open(file_name, "r") as file:
    death_stock_list = [line.strip() for line in file if line.strip()]

death_to_del_stock_list = []
death_to_del_stock_list = common.Final_process.process_stocks(death_stock_list, mode="week_death")

Processed stock codes saved to ./txt_lib/processed_KDJ日死.txt.


In [9]:
## 处理 KDJ日金 + 日死 → KDJ日死
gold_to_move_day_death_stock_list = []
gold_to_move_day_death_stock_list = common.Final_process.process_stocks(gold_stock_list, mode="daily_death")

In [10]:
## 处理 KDJ日金 + Bol → 操作1.1
gold_to_1_1_list = []
gold_to_1_1_list = common.Final_process.process_stocks(gold_stock_list, mode="bol")

In [11]:
## 处理 KDJ日金 + J线拐头 → KDJ8
gold_to_move_8_stock_list = []
gold_to_move_8_stock_list = common.Final_process.process_stocks(
    gold_stock_list, mode="J_turn_around"
)

In [ ]:
# output_file = "./txt_lib/processed_KDJ日死.txt"
# # add_capital(input_file, output_file)

# file_name = output_file
# with open(file_name, "r") as file:
#     death_stock_list = [line.strip() for line in file if line.strip()]

In [7]:
## KDJ日死 + Bol: 前次碰上轨，今日碰下轨 → 操作2
death_to_op2_list = []
death_to_op2_list = common.Final_process.process_stocks(death_stock_list, mode="bol_lower")

In [4]:
## 找出 KDJ日死 接近金叉的股票
death_near_gold_list = []
death_near_gold_list = common.Final_process.process_stocks(
    death_stock_list, mode="kdj_near_gold"
)

# 图片 Excel生成

In [14]:
try:
    folder = Path("./output")
    folder.mkdir()
except Exception as e:
    pass

try:
    folder = Path("./output/KDJ日金")
    folder.mkdir()
    folder = Path("./output/KDJ日死")
    folder.mkdir()
    folder = Path("./output/KDJ8")
    folder.mkdir()
    folder = Path("./output/操作1.1")
    folder.mkdir()
    folder = Path("./output/操作2")
    folder.mkdir()
except Exception as e:
    pass

In [30]:
# 今日金叉数据
today_gold_txt_path = "./output/golden_output/today_gold_cross.txt"
today_gold_list = []
with open(today_gold_txt_path, "r") as file:
    for line in file:
        stock_code = line.strip()
        today_gold_list.append(stock_code)

# KDJ日金 文件生成
kdj_res = []
for stock_code in gold_stock_list:
    condition1 = stock_code not in gold_to_del_stock_list # 周死
    condition2 = stock_code not in gold_to_move_day_death_stock_list # 日死
    condition3 = stock_code not in gold_to_move_8_stock_list #拐头
    if condition1 and condition2 and condition3:
        kdj_res.append(stock_code)

# 加入今日金叉数据
for stock_code in today_gold_list:
    if stock_code not in kdj_res:
        kdj_res.append(stock_code)

output_file = './output/KDJ日金/KDJ日金.xlsx'
common.Final_file.output_excel(kdj_res,output_file,condition='日金')
common.Final_file.mk_pic(output_file)  # 生成所有图片

Excel 文件已生成: ./output/KDJ日金/KDJ日金.xlsx
所有图片已生成！


In [31]:
# 覆写KDJ日金txt
output_file = "./txt_lib/KDJ日金.txt"
with open(output_file, "w") as outfile1:
    for stock_code in kdj_res:
        processed_code = stock_code[2:]
        outfile1.write(processed_code + "\n")

print(f"Processed stock codes saved to {output_file}.")

Processed stock codes saved to ./txt_lib/KDJ日金.txt.


In [17]:
# KDJ8 文件生成

# 读取昨日kdj8 txt
yest_kdj8_list = []
yest_kdj8_path = "./txt_lib/today_kdj8.txt"
with open(yest_kdj8_path, "r") as file:
    for line in file:
        stock_code = line.strip()
        if stock_code.startswith("6"):
            processed_code = f"sh{stock_code}"
        elif stock_code.startswith("0"):
            processed_code = f"sz{stock_code}"
        else:
            processed_code = stock_code
        yest_kdj8_list.append(processed_code)

# 筛出日死代码
kdj8_to_move_day_death_stock_list = []
kdj8_to_move_day_death_stock_list = common.Final_process.process_stocks(
    yest_kdj8_list, mode="daily_death"
)

# 筛出周死代码
kdj8_to_del_stock_list = []
kdj8_to_del_stock_list = common.Final_process.process_stocks(
    yest_kdj8_list, mode="week_death"
)

today_kdj8 = []

for stock_code in yest_kdj8_list:
    condition1 = stock_code not in kdj8_to_del_stock_list # 周死
    condition2 = stock_code not in kdj8_to_move_day_death_stock_list # 日死
    condition3 = stock_code not in gold_to_move_8_stock_list # 今日
    if condition1 and condition2 and condition3:
        today_kdj8.append(stock_code)

today_kdj8 += gold_to_move_8_stock_list

# 最终输出
output_file = "./output/KDJ8/KDJ8.xlsx"
common.Final_file.output_excel(today_kdj8, output_file, condition="8")
common.Final_file.mk_pic(output_file)  # 生成所有图片

# 生成今日 KDJ8 TXT
today_kdj8_txt_path = "./txt_lib/today_kdj8.txt"
with open(today_kdj8_txt_path, "w") as file:
    for item in today_kdj8:
        file.write(item + "\n")  # 每个元素后加换行符

print("今日KDJ8 txt文件已生成。")

Excel 文件已生成: ./output/KDJ8/KDJ8.xlsx
所有图片已生成！
今日KDJ8 txt文件已生成。


In [18]:
# KDJ日死 文件生成
kdj_death = []

# 今日死叉数据
today_death_txt_path = "./output/death_output/today_death_cross.txt"
today_death_list = []
with open(today_death_txt_path, "r") as file:
    for line in file:
        stock_code = line.strip()
        today_death_list.append(stock_code)

for stock_code in death_stock_list:
    condition1 = stock_code not in death_to_del_stock_list
    condition2 = stock_code not in death_to_op2_list
    if condition1 and condition2:
        kdj_death.append(stock_code)

# 加入今日死叉数据
for stock_code in today_death_list:
    if stock_code not in kdj_death:
        kdj_death.append(stock_code)
        
# 加入今日kdj8日死数据
for stock_code in kdj8_to_move_day_death_stock_list:
    if stock_code not in kdj_death:
        kdj_death.append(stock_code)

output_file = "./output/KDJ日死/KDJ日死.xlsx"
common.Final_file.output_excel(kdj_death, output_file, condition="日死")
common.Final_file.mk_pic(output_file)  # 生成所有图片

Excel 文件已生成: ./output/KDJ日死/KDJ日死.xlsx
所有图片已生成！


In [ ]:
# 输出操作2 Excel 图片
death_to_op2_list += death_near_gold_list

output_file = "./output/操作2/操作2.xlsx"
common.Final_file.output_excel(death_to_op2_list, output_file, condition="bol")
common.Final_file.mk_pic(output_file)  # 生成所有图片

Excel 文件已生成: ./output/操作2/操作2.xlsx
所有图片已生成！


In [20]:
# 覆写KDJ日死txt
output_file = "./txt_lib/KDJ日死.txt"
with open(output_file, "w") as outfile1:
    for stock_code in kdj_death:
        processed_code = stock_code[2:]
        outfile1.write(processed_code + "\n")

print(f"Processed stock codes saved to {output_file}.")

Processed stock codes saved to ./txt_lib/KDJ日死.txt.


In [21]:
# 输出操作1.1 Excel 图片
op1_1 = []
op1_1 = to_perfect_stock_list + gold_to_1_1_list
output_file = "./output/操作1.1/操作1.1.xlsx"

common.Final_file.output_excel(op1_1, output_file, condition="bol")
common.Final_file.mk_pic(output_file)  # 生成所有图片

Excel 文件已生成: ./output/操作1.1/操作1.1.xlsx
所有图片已生成！


## TEST

In [25]:
import calculate
import pandas as pd

current_date = datetime.now()
end_date = current_date

stock_code = "sh603681"

df = ak.stock_zh_a_daily(
    symbol=stock_code,
    start_date="2010-01-01",
    end_date=end_date.strftime("%Y-%m-%d"),
    adjust="qfq",
)

daily_kdj = calculate.KDJ.calculate_kdj(df).tail(7)
# # 预处理数据
# df.index = pd.to_datetime(df["date"])
# df = df.sort_index()
# df = calculate.Bol.calculate_bollinger_bands(df)
# touch_records = calculate.Bol.detect_bollinger_touch(df)  # 获取碰轨事件

In [26]:
df["J"].iloc[-1] < df["K"].iloc[-1]

np.True_

In [2]:
stock_code ="sh600918"
current_date = datetime.now()
df = ak.stock_zh_a_daily(
    symbol=stock_code,
    start_date="2015-01-01",
    end_date=current_date.strftime("%Y-%m-%d"),
    adjust="qfq",
)  # qfq=前复权

df = calculate.KDJ.calculate_kdj(df)

condition1 = (df["K"].iloc[-2] - df["D"].iloc[-2]) < (df["K"].iloc[-1] - df["D"].iloc[-1])# 昨日KD差值大于今日
condition2 = df["K"].iloc[-1] < df["D"].iloc[-1] # 今日K < D
condition3 = ((df["D"].iloc[-1] - df["J"].iloc[-1]) < 12) # 今日D J 差值
condition4 = (df["J"].iloc[-1] - df["J"].iloc[-2]) > 15 # 今昨 J 差值